In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Giải nén dataset trong Colab
import zipfile
import os
zip_path = '/content/drive/MyDrive/datasets/archive.zip'
extract_path = '/content/archive'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Giải nén xong!")

Giải nén xong!


In [5]:
# Cài thư viện cần thiết
!pip install tensorflow matplotlib scikit-learn


**Load & Chuẩn bị dữ liệu**

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Đường dẫn folder train/test
train_dir = '/content/archive/asl_alphabet_train/asl_alphabet_train'
test_dir = '/content/archive/asl_alphabet_test/asl_alphabet_test'

# Chuẩn bị generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


**Xây dựng CNN Model**

In [7]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 686,941 (2.62 MB)

 Trainable params: 686,941 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

**Train model**

In [8]:
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2175/2175 ━━━━━━━━━━━━━━━━━━━━ 80s 34ms/step - accuracy: 0.3992 - loss: 2.0083 - val_accuracy: 0.7153 - val_loss: 0.9397
Epoch 2/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 65s 30ms/step - accuracy: 0.8821 - loss: 0.3379 - val_accuracy: 0.7690 - val_loss: 0.8944
Epoch 3/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - accuracy: 0.9388 - loss: 0.1733 - val_accuracy: 0.7800 - val_loss: 0.9985
Epoch 4/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 72s 33ms/step - accuracy: 0.9608 - loss: 0.1136 - val_accuracy: 0.8082 - val_loss: 0.9793
Epoch 5/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 81s 33ms/step - accuracy: 0.9698 - loss: 0.0889 - val_accuracy: 0.7687 - val_loss: 0.9973
Epoch 6/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 65s 30ms/step - accuracy: 0.9767 - loss: 0.0693 - val_accuracy: 0.8262 - val_loss: 0.8977
Epoch 7/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 72s 33ms/step - accuracy: 0.9810 - loss: 0.0568 - val_accuracy: 0.8371 - val_loss: 0.8025
Epoch 8/10
2175/2175 ━━━━━━━━━━━━━━━━━━━━ 64s 29ms/step - accuracy: 0.9828 - loss: 0.05

**Lưu model về Google Drive**

In [9]:
model.save('/content/drive/MyDrive/asl_cnn_model.h5')
print("Model đã lưu vào Google Drive!")

Model đã lưu vào Google Drive!


**Test model với ảnh**

In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = '/content/archive/asl_alphabet_test/asl_alphabet_test/E_test.jpg'  # đổi thành ảnh test
img = image.load_img(img_path, target_size=(64,64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.0

pred = model.predict(x)
class_index = np.argmax(pred)
class_labels = list(train_data.class_indices.keys())

print("Dự đoán:", class_labels[class_index])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Dự đoán: E
